# DECIDE – Pipeline LLM para classificação de queries (Groq API)

Este notebook replica o ficheiro `pipeline_groupA.py` mas em formato interativo,
para podermos:
- testar cada passo,
- inspecionar DataFrames,
- alterar parâmetros (modelo, batch size, prompt, etc.).

### Ambiente recomendado

```bash
mamba create -n decide_env python=3.10
mamba activate decide_env
pip install groq pandas python-dotenv openpyxl perplexityai


In [ ]:
# IMPORTS

import os
import time
import json
import re
import unicodedata
import pandas as pd

from dotenv import load_dotenv
from datetime import datetime
from perplexity import Perplexity

import logging
from logging.handlers import RotatingFileHandler

# ============================================================
# LOGGING CONFIGURATION
# ============================================================

log_filename = datetime.now().strftime("pipeline_run_%Y-%m-%d_%H-%M-%S.log")


log_handler = RotatingFileHandler(
    log_filename,
    maxBytes=5_000_000,
    backupCount=3,
    encoding="utf-8"
)

formatter = logging.Formatter(
    fmt="%(asctime)s | %(levelname)s | %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)

log_handler.setFormatter(formatter)

logger = logging.getLogger("DECIDE_PIPELINE")
logger.setLevel(logging.INFO)
logger.addHandler(log_handler)

console_handler = logging.StreamHandler()
console_handler.setFormatter(formatter)
logger.addHandler(console_handler)

# ============================================================
# CONFIGURAR PERPLEXITY API
# ============================================================

load_dotenv()

api_key = os.environ.get("PERPLEXITY_API_KEY")
if not api_key:
    raise ValueError("Variável de ambiente PERPLEXITY_API_KEY não definida.")

client = Perplexity(api_key=api_key)

MODEL_NAME = "sonar"   # ou "sonar-pro"
BATCH_SIZE = 50

logger.info("Ambiente carregado e cliente Perplexity SDK configurado!")


In [2]:
# ============================================================
# NORMALIZAÇÃO DE QUERIES
# ============================================================

def normalize_query(q):
    """Remove espaços invisíveis + normaliza Unicode + limpa whitespace."""
    if pd.isna(q):
        return ""
    q = str(q)

    q = unicodedata.normalize("NFKC", q)    # normalização Unicode
    q = q.replace("\u200b", "")             # zero-width space
    q = q.replace("\xa0", " ")              # NBSP
    q = " ".join(q.split())                 # remover múltiplos espaços / trim

    return q


In [ ]:
# ============================================================
# 1. LER FICHEIRO EXCEL
# ============================================================

def load_queries(path="queries_middle_east_test.xlsx"):
    df = pd.read_excel(path)
    # df = df.tail(5)  # PARA TESTES RÁPIDOS

    if "Query" not in df.columns:
        raise ValueError("A coluna 'Query' não existe no ficheiro.")

    # Remover linhas com queries vazias
    df = df.dropna(subset=["Query"]).copy()

    # print('Antes')
    # print(df["Query"])
    # print('---')
    # Normalização das queries
    df["Query"] = df["Query"].apply(normalize_query)
    # print('Depois')
    # print(df["Query"])

    # df["QueryID"] = range(1, len(df) + 1)
    return df

# TESTE: carregar e espreitar
df = load_queries()
logger.info(f"Total de linhas: {len(df)}")
df.head()


In [20]:
# ============================================================
# 2. REMOVER DUPLICADOS POR TEXTO
# ============================================================

def deduplicate_queries(df):
    df_unique = df[["Query"]].drop_duplicates().reset_index(drop=True)
    df_unique["UniqueID"] = range(1, len(df_unique) + 1)
    return df_unique

df_unique = deduplicate_queries(df)
logger.info(f"Queries únicas: {len(df_unique)}")
df_unique.head()


2025-12-03 10:20:04 | INFO | Queries únicas: 293


,Query,UniqueID
0,antihistaminique,1
1,antihistaminiques,2
2,مضادات الهيستامين,3
3,les antihistaminiques,4
4,anti histaminique,5


In [ ]:
# ============================================================
# 3. BATCHING
# ============================================================

def chunk_list(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

In [ ]:
# ============================================================
# 4. PROMPT — SUPPLEMENT BOX 2A (COM JSON)
# ============================================================

def build_prompt_for_batch(batch):
    text = (
        "We have extracted several queries from GoogleTrends.\n\n"
        "We want to identify what are the queries which explicitly convey a question. "
        "These queries can be in different languages, including English, Arabic, "
        "French, Persian (Farsi), Turkish, Russian, Spanish, German or Dutch.\n\n"
        "Below, you can find the list of queries.\n\n"
        "Return ONLY a JSON array, with no explanations or additional text. "
        "Each element must have the form:\n"
        "{ \"query\": \"<query text>\", \"explicit_question\": \"YES\" or \"NO\" }\n\n"
        "Write \"YES\" only if the query explicitly conveys a question. "
        "Otherwise, write \"NO\".\n\n"
        "List of queries to be classified:\n"
    )

    for q in batch:
        text += f"- {q}\n"

    return text

# ============================================================
# 5. FUNÇÃO PARA EXTRAIR JSON DA RESPOSTA DO LLM
# ============================================================

def safe_json_extract(text):
    if not text:
        return None

    cleaned = text.strip()

    # 1) Strip leading markdown fence like ```json
    if cleaned.startswith("```"):
        cleaned = re.sub(r"^```[a-zA-Z]*\s*", "", cleaned)

    # 2) Strip trailing fence ```
    if cleaned.endswith("```"):
        cleaned = re.sub(r"\s*```$", "", cleaned)

    # 3) Extract the first JSON array
    match = re.search(r"\[.*\]", cleaned, re.DOTALL)
    if not match:
        return None

    candidate = match.group(0).strip()

    try:
        return json.loads(candidate)
    except json.JSONDecodeError:
        return None


In [23]:
test_prompt = build_prompt_for_batch(df_unique["Query"].head(3).tolist())
print(test_prompt)


We have extracted several queries from GoogleTrends from the following nine countries: Algeria, Egypt, Iran, Iraq, Morocco, Pakistan, Saudi Arabia, Turkey and the United Arab Emirates.

We want to identify which queries explicitly convey a question. These queries can be in English, Portuguese, French, Spanish or in any language spoken in the aforementioned countries.

Below, you can find the list of queries.

For each query, return a JSON array where each element has the form:
{ "query": "<query text>", "explicit_question": "YES" or "NO" }

Write "YES" only if the query explicitly conveys a question. Otherwise, write "NO".

List of queries to be classified:
- antihistaminique
- antihistaminiques
- مضادات الهيستامين



In [ ]:
# ============================================================
# 6. FUNÇÃO PARA CLASSIFICAR UM BATCH COM O LLM
# ============================================================
def classify_batch_with_llm(batch):
    """
    Devolve lista de dicts: { "query": ..., "explicit_question": "YES"/"NO" }
    """
    prompt = build_prompt_for_batch(batch)

    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role": "user", "content": prompt}],
        temperature=0  # respostas mais determinísticas, menos aleatórias
    )

    raw = response.choices[0].message.content
    # print(raw)  # para debug

    data = safe_json_extract(raw)
    if data is None:
        logger.error("Falha ao extrair JSON; a guardar resposta bruta para debug.")
        with open("failed_batch.txt", "a", encoding="utf-8") as f:
            f.write(raw + "\n\n" + "="*80 + "\n\n")
        return []

    cleaned = []
    for item in data:
        cleaned.append({
            "query": item.get("query", "").strip(),
            "explicit_question": item.get("explicit_question", "NO").strip().upper()
        })

    return cleaned

# TESTE COM UM MINI-BATCH
mini_batch = df_unique["Query"].head(5).tolist()
test_output = classify_batch_with_llm(mini_batch)
test_output


[{'query': 'antihistaminique', 'explicit_question': 'NO'},
 {'query': 'antihistaminiques', 'explicit_question': 'NO'},
 {'query': 'مضادات الهيستامين', 'explicit_question': 'NO'},
 {'query': 'les antihistaminiques', 'explicit_question': 'NO'},
 {'query': 'anti histaminique', 'explicit_question': 'NO'}]

In [25]:
# ============================================================
# 7. FUNÇÃO: EXECUTAR UMA RUN COMPLETA (RUN 1 / RUN 2)
# ============================================================

def run_llm_classification(df_unique, run_name):
    rows = df_unique[["UniqueID", "Query"]]
    results = []

    for batch_df in chunk_list(rows, BATCH_SIZE):
        logger.info(f"{run_name} - batch com {len(batch_df)} queries...")

        batch_queries = batch_df["Query"].tolist()
        batch_ids = batch_df["UniqueID"].tolist()

        out = classify_batch_with_llm(batch_queries)

        for uid, item in zip(batch_ids, out):
            results.append({
                "UniqueID": uid,
                run_name: item["explicit_question"]
            })

        time.sleep(1)

    return pd.DataFrame(results)

# ⚠️ Para testes usa um subset:
df_unique_test = df_unique.head(20).copy()
df_run1_test = run_llm_classification(df_unique_test, "LLM_run1")
df_run1_test.head()


2025-12-03 10:20:49 | INFO | LLM_run1 - batch com 20 queries...


,UniqueID,LLM_run1
0,1,NO
1,2,NO
2,3,NO
3,4,NO
4,5,NO


In [ ]:
# ============================================================
# 8. CLASSIFICAÇÃO POR REGRAS MULTILINGUE
# ============================================================

# 1) Interrogative keywords
QUESTION_KEYWORDS = [
    # English
    "what", "when", "where", "why", "how", "who", "whom", "which",
    "do", "did", "does", "are", "is", "can", "could", "should", "would",

    # French
    "quoi", "quand", "où", "pourquoi", "comment", "qui", "lequel",
    "est-ce", "peux-tu", "pourrais-tu",

    # Spanish
    "qué", "cuándo", "dónde", "por qué", "cómo", "quién", "cuál", "puedes", "podrías",

    # German
    "was", "wann", "wo", "warum", "wie", "wer", "welche", "kann", "könnte",

    # Dutch
    "wat", "wanneer", "waar", "waarom", "hoe", "wie", "welke", "kan", "zou",

    # Russian
    "что", "когда", "где", "почему", "как", "кто", "который", "может", "могли бы",

    # Arabic
    "ماذا", "متى", "أين", "لماذا", "كيف", "من", "هل", "أيمكن",

    # Persian
    "چه", "کی", "کجا", "چرا", "چطور", "کیست", "آیا",

    # Turkish
    "ne", "ne zaman", "nerede", "neden", "nasıl", "kim", "hangi", "mı", "mi", "mu", "mü"
]

# 2) Structural patterns
STRUCTURAL_PATTERNS = [
    r".*\?\s*$",                                  # explicit '?'
    r"^(can|could|should|would|do|did|does)\b",   # English inversion
    r"^(is|are|was|were|am)\b",                   # English BE inversion
    r".*\b(mı|mi|mu|mü)\?$",                      # Turkish question particle
    r"^[^.!?]*\b(est-ce que)\b",                  # French
    r"^[^.!?]*\b(هل)\b",                          # Arabic
    r"^[^.!?]*\b(آیا)\b",                         # Persian
]

# 3) Implicit question markers
IMPLICIT_PATTERNS = [
    r"could you\b.*",
    r"would you\b.*",
    r"can you\b.*",
    r"please explain\b.*",
    r"i wonder if\b.*",
    r"i would like to know\b.*"
]


def is_question_multilingual(sentence: str) -> bool:
    if not sentence:
        return False

    s = sentence.strip().lower()

    # Rule 1: punctuation
    if re.search(r".*\?\s*$", s):
        return True

    # Rule 2: structural patterns
    for p in STRUCTURAL_PATTERNS:
        if re.search(p, s):
            return True

    # Rule 3: keyword-based detection
    for kw in QUESTION_KEYWORDS:
        if re.search(rf"\b{re.escape(kw)}\b", s):
            return True

    # Rule 4: implicit patterns
    for p in IMPLICIT_PATTERNS:
        if re.search(p, s):
            return True

    return False


def apply_multilingual_rules(df_unique):
    df_unique["Rules"] = df_unique["Query"].apply(
        lambda x: "YES" if is_question_multilingual(x) else "NO"
    )
    return df_unique

# TESTE
df_unique_test = apply_multilingual_rules(df_unique.head(20).copy())
df_unique_test.head()


,Query,UniqueID,Rules
0,antihistaminique,1,NO
1,antihistaminiques,2,NO
2,مضادات الهيستامين,3,NO
3,les antihistaminiques,4,NO
4,anti histaminique,5,NO


In [ ]:
# ============================================================
# 9. MERGE FINAL
# ============================================================

def merge_results(df_original, df_unique, df_run1, df_run2):
    temp = df_unique.merge(df_run1, on="UniqueID", how="left")
    temp = temp.merge(df_run2, on="UniqueID", how="left")

    df_final = df_original.merge(
        temp[["Query", "Rules", "LLM_run1", "LLM_run2"]],
        on="Query",
        how="left"
    )

    return df_final

# Exemplo de pipeline completo em modo “manual”:

start_time = time.time()
logger.info("=== PIPELINE COMPLETO (TESTE EM SUBSET) ===")

df = load_queries()
df_unique = deduplicate_queries(df)

df_unique.to_excel("unique_test.xlsx", index=False)

# Para testes, subset:
df_unique_small = df_unique.tail(20).copy()

df_run1 = run_llm_classification(df_unique, "LLM_run1")
df_run1.to_excel("df_run1_test.xlsx", index=False)

df_unique_shuffled = df_unique.sample(frac=1, random_state=None).reset_index(drop=True)  # sample() função pandas usada para escolher linhas de forma aleatória
df_run2 = run_llm_classification(df_unique_shuffled, "LLM_run2")
# df_run2 = run_llm_classification(df_unique, "LLM_run2")
df_run2.to_excel("df_run2_test.xlsx", index=False)
df_unique = apply_multilingual_rules(df_unique)
df_unique.to_excel("df_unique_small_test.xlsx", index=False)

df_final_test = merge_results(df, df_unique, df_run1, df_run2)
df_final_test.to_excel("df_final_test.xlsx", index=False)
df_final_test.head()

elapsed = int(time.time() - start_time)
logger.info(f"Tempo total (subset): {elapsed} segundos")
